# POLICE DEPARTMENT DATA

In [3]:
# setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import pprint

# Import API key
from API_Keys import fbi_key

## List of Police Departments Used with ORI Data
#### ORI Data was pulled from the following website:
https://www.icpsr.umich.edu/files/NACJD/ORIs/STATESoris.html

In [4]:
# List of PD stations
police_list = []
police_info = {"BOSTON POLICE DEPARTMENT": "MA0130100",
               "CHARLOTTESVILLE POLICE DEPT": "VA1020000",
               "CHATTANOOGA POLICE DEPARTMENT": "TN0330100",
               "CHICAGO POLICE DEPT":"ILCPD0000",
               "DALLAS POLICE DEPARTMENT": "TXDPD0000",
               "DENVER POLICE DEPARTMENT":"CODPD0000",
               "KANSAS CITY POLICE DEPARTMENT":"MOKPD0000",
               "LAS VEGAS METRO POLICE DEPARTMENT": "NV0020100",
               "NEW ORLEANS POLICE DEPARTMENT":"LANPD0000",
               "PORTLAND POLICE BUREAU":"OR0260200",
               "SAN FRANCISCO POLICE DEPARTMENT": "CA0380100",
               "SIOUX FALLS POLICE DEPARTMENT": "SD0490200"}

police_list.append(police_info)
police_stations = pd.DataFrame(police_list).T
police_stations.columns = ["ORI"]
police_stations

,ORI
BOSTON POLICE DEPARTMENT,MA0130100
CHARLOTTESVILLE POLICE DEPT,VA1020000
CHATTANOOGA POLICE DEPARTMENT,TN0330100
CHICAGO POLICE DEPT,ILCPD0000
DALLAS POLICE DEPARTMENT,TXDPD0000
DENVER POLICE DEPARTMENT,CODPD0000
KANSAS CITY POLICE DEPARTMENT,MOKPD0000
LAS VEGAS METRO POLICE DEPARTMENT,NV0020100
NEW ORLEANS POLICE DEPARTMENT,LANPD0000
PORTLAND POLICE BUREAU,OR0260200


In [5]:
#list of ORIs for the API Call
ori_list = police_stations["ORI"].tolist()
ori_list

['MA0130100',
 'VA1020000',
 'TN0330100',
 'ILCPD0000',
 'TXDPD0000',
 'CODPD0000',
 'MOKPD0000',
 'NV0020100',
 'LANPD0000',
 'OR0260200',
 'CA0380100',
 'SD0490200']

### The following website was used for pulling crime data
https://crime-data-explorer.fr.cloud.gov/api

Two different API call urs are used 
One lists the summarized offenses per agency, and the other will give only the total for the specific offense per agency.  
"https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/{ori}/offenses/2019/2019?API_KEY={fbi_key}"  
"https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/{ori}/{offense}/2019/2019?API_KEY={fbi_key}"  
The urls are essentially the same except the second has offenses as a variable that can be changed  

Per the website when testing out the url for specific offenses there are only ten options, but when pulling all offenses for the agency there are 12.  
Human trafficing isn't included as an option on the website to test, but it still can be pulled.   
Rape-Legacy also isn't included as an option on the website to test, but it still can be pulled.  
Rape-Legacy and Rape are both found when running the url specifically rape, but rape-legacy pulls only rape-legacy.  

In [6]:
#the following url is for a list of all offenses per ORI
ori = "MOKPD0000"

url = f"https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/{ori}/offenses/2019/2019?API_KEY={fbi_key}"
response = requests.get(url).json()

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(response)

{   'pagination': {'count': 12, 'page': 0, 'pages': 1, 'per_page': 0},
    'results': [   {   'actual': 1443,
                       'cleared': 253,
                       'data_year': 2019,
                       'offense': 'robbery',
                       'ori': 'MOKPD0000',
                       'state_abbr': 'MO'},
                   {   'actual': 7308,
                       'cleared': 1829,
                       'data_year': 2019,
                       'offense': 'violent-crime',
                       'ori': 'MOKPD0000',
                       'state_abbr': 'MO'},
                   {   'actual': 5159,
                       'cleared': 1320,
                       'data_year': 2019,
                       'offense': 'aggravated-assault',
                       'ori': 'MOKPD0000',
                       'state_abbr': 'MO'},
                   {   'actual': 160,
                       'cleared': 16,
                       'data_year': 2019,
                       'offense': 'a

In [7]:
#from the result above, we can see there are 12 listed offenses, making a list of each offense
offense_type = []
ori = "MOKPD0000"

url = f"https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/{ori}/offenses/2019/2019?API_KEY={fbi_key}"
response = requests.get(url).json()

offense_type.append(response["results"][0]["offense"])
offense_type.append(response["results"][1]["offense"])
offense_type.append(response["results"][2]["offense"])
offense_type.append(response["results"][3]["offense"])
offense_type.append(response["results"][4]["offense"])
offense_type.append(response["results"][5]["offense"])
offense_type.append(response["results"][6]["offense"])
offense_type.append(response["results"][7]["offense"])
offense_type.append(response["results"][8]["offense"])
offense_type.append(response["results"][9]["offense"])
offense_type.append(response["results"][10]["offense"])
offense_type.append(response["results"][11]["offense"])

offense_type

['robbery',
 'violent-crime',
 'aggravated-assault',
 'arson',
 'burglary',
 'homicide',
 'human-trafficing',
 'larceny',
 'property-crime',
 'motor-vehicle-theft',
 'rape-legacy',
 'rape']

In [8]:
#dropping rape-legacy from list as rape can pull both
offense_type.remove("rape-legacy")
offense_type

['robbery',
 'violent-crime',
 'aggravated-assault',
 'arson',
 'burglary',
 'homicide',
 'human-trafficing',
 'larceny',
 'property-crime',
 'motor-vehicle-theft',
 'rape']

In [9]:
#API Call for all offenses for all ORIs
offenses = []
actual = []
state = []
ori_name = []

for ori in ori_list:
    for offense in offense_type:
        offense_url = f"https://api.usa.gov/crime/fbi/sapi/api/summarized/agencies/{ori}/{offense}/2019/2019?API_KEY={fbi_key}"
        response = requests.get(offense_url).json()
        
        #rape has two results and both must be printed 
        #(each department switches whether its return 0 or 1, both must be pulled)
        if offense == "rape":
            try:
                ori_name.append(response["results"][0]["ori"]) 
                offenses.append(response["results"][0]["offense"])
                state.append(response["results"][0]["state_abbr"])
                actual.append(response["results"][0]["actual"])
                ori_name.append(response["results"][1]["ori"])
                offenses.append(response["results"][1]["offense"])                  
                state.append(response["results"][1]["state_abbr"])
                actual.append(response["results"][1]["actual"])
            
            except:
                print("not found")
        
        #all other offenses have one results to print
        else:
            try:
                ori_name.append(response["results"][0]["ori"])
                offenses.append(response["results"][0]["offense"])
                state.append(response["results"][0]["state_abbr"])
                actual.append(response["results"][0]["actual"])
            except:
                print("not found")

## All Crime Stats DataFrame

In [11]:
#make a data frame of all the information from the API call
crime_data_all={"ORI":ori_name,"State":state,"Offense":offenses,"Crimes Committed":actual}
crime_all_df = pd.DataFrame(crime_data_all)
crime_all_df

,ORI,State,Offense,Crimes Committed
0,MA0130100,MA,robbery,1039
1,MA0130100,MA,violent-crime,4284
2,MA0130100,MA,aggravated-assault,2932
3,MA0130100,MA,arson,4
4,MA0130100,MA,burglary,1703
...,...,...,...,...
139,SD0490200,SD,larceny,4300
140,SD0490200,SD,property-crime,5653
141,SD0490200,SD,motor-vehicle-theft,658
142,SD0490200,SD,rape,116


### Sample individual DataFrame

In [12]:
#sample results for one police department
sample = crime_all_df.loc[crime_all_df["ORI"]=="MA0130100"]
sample

,ORI,State,Offense,Crimes Committed
0,MA0130100,MA,robbery,1039
1,MA0130100,MA,violent-crime,4284
2,MA0130100,MA,aggravated-assault,2932
3,MA0130100,MA,arson,4
4,MA0130100,MA,burglary,1703
5,MA0130100,MA,homicide,42
6,MA0130100,MA,human-trafficing,0
7,MA0130100,MA,larceny,10590
8,MA0130100,MA,property-crime,13213
9,MA0130100,MA,motor-vehicle-theft,920


## Police Department DataFrames

### Separating each department from the dataframe and setting up the data as desired

Of note, the human-trafficking category is 0 in every ori we used, so that data was dropped. The rape-legacy should be included in list of rape per the websites documentation, and can be dropped as well

Please note the difference between rape and rape-legacy is that rape-legacy is an older definition of rape. Rape as a category includes that and more rape crimes that would include other sex crimes that were not included in the older definition. The rape-legacy is still included for statistical information in the databases, but it is not needed for this data set. The rape category will be renamed to sex crimes to be more inclusive for understanding the data provided.

Also adding sums of categories
There are three totals/sums taken.
There is the sum overall for all crime, a sum of property crime which includes: arson, auto theft, burglary, larcency, property crime, and robbery. The third sum is for person crimes which includes: agg assault, homicide, sex crimes, and violent crime. 

In [13]:
#station 1 of 12
#BOSTON POLICE DEPARTMENT
#MA0130100

#department dataframe
ma_crime = crime_all_df.loc[crime_all_df["ORI"]=="MA0130100"]

#changing index to offense and removing title
ma_crime = ma_crime.set_index("Offense")
ma_crime.index.name = None

#dropping human_trafficking & rape categoties
ma_crime = ma_crime.drop("human-trafficing")
ma_crime = ma_crime.drop("rape-legacy")

#get total number of crimes for the department
total_ma = ma_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
ma_crime = pd.DataFrame(ma_crime).T

#removing rows as it is duplicating data
ma_crime = ma_crime.drop("State")
ma_crime = ma_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_ma = (ma_crime["aggravated-assault"]["Crimes Committed"] + ma_crime["homicide"]["Crimes Committed"] 
            + ma_crime["rape"]["Crimes Committed"] + ma_crime["violent-crime"]["Crimes Committed"])
property_ma = (ma_crime["arson"]["Crimes Committed"] + ma_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + ma_crime["burglary"]["Crimes Committed"] + ma_crime["larceny"]["Crimes Committed"] 
            + ma_crime["property-crime"]["Crimes Committed"] + ma_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
ma_crime["ORI"] = "MA0130100"
ma_crime["State"] = "MA"
ma_crime["PD Name"] = "BOSTON POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
ma_crime = ma_crime.set_index("PD Name")
ma_crime.index.name = None

#sorting columns
ma_crime = ma_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
ma_crime = ma_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
ma_crime["Total Crime"] = total_ma
#sum of all person crimes
ma_crime["Total Person Crime"] = person_ma
#sum of all property crimes
ma_crime["Total Property Crime"] = property_ma

ma_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
BOSTON POLICE DEPARTMENT,MA,MA0130100,2932,4,920,1703,42,10590,13213,1039,231,4284,34958,7489,27469


In [14]:
#making second data frame transposed 
ma_crime2 = pd.DataFrame(ma_crime).T

#sending both to csv files so both options are available
ma_crime.to_csv("CSV_Files/PD_Crime_Data/MA0130100.csv", encoding="utf-8")
ma_crime2.to_csv("CSV_Files/PD_Crime_Data/MA0130100-2.csv", encoding="utf-8")

In [15]:
#station 2 of 12
#CHARLOTTESVILLE POLICE DEPT
#VA1020000

#department dataframe
va_crime = crime_all_df.loc[crime_all_df["ORI"]=="VA1020000"]

#changing index to offense and removing title
va_crime = va_crime.set_index("Offense")
va_crime.index.name = None

#dropping human_trafficking & rape categoties
va_crime = va_crime.drop("human-trafficing")
va_crime = va_crime.drop("rape-legacy")

#get total number of crimes for the department
total_va = va_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
va_crime = pd.DataFrame(va_crime).T

#removing rows as it is duplicating data
va_crime = va_crime.drop("State")
va_crime = va_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_va = (va_crime["aggravated-assault"]["Crimes Committed"] + va_crime["homicide"]["Crimes Committed"] 
            + va_crime["rape"]["Crimes Committed"] + va_crime["violent-crime"]["Crimes Committed"])
property_va = (va_crime["arson"]["Crimes Committed"] + va_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + va_crime["burglary"]["Crimes Committed"] + va_crime["larceny"]["Crimes Committed"] 
            + va_crime["property-crime"]["Crimes Committed"] + va_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
va_crime["ORI"] = "VA1020000"
va_crime["State"] = "VA"
va_crime["PD Name"] = "CHARLOTTESVILLE POLICE DEPT" 

#make the PD name the index and dropping index name
va_crime = va_crime.set_index("PD Name")
va_crime.index.name = None

#sorting columns
va_crime = va_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
va_crime = va_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
va_crime["Total Crime"] = total_va
#sum of all person crimes
va_crime["Total Person Crime"] = person_va
#sum of all property crimes
va_crime["Total Property Crime"] = property_va

va_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
CHARLOTTESVILLE POLICE DEPT,VA,VA1020000,103,5,78,122,0,924,1124,22,32,181,2591,316,2275


In [16]:
#making second data frame transposed 
va_crime2 = pd.DataFrame(va_crime).T

#sending both to csv files so both options are available
va_crime.to_csv("CSV_Files/PD_Crime_Data/VA1020000.csv", encoding="utf-8")
va_crime2.to_csv("CSV_Files/PD_Crime_Data/VA1020000-2.csv", encoding="utf-8")

In [17]:
#station 3 of 12
#CHATTANOOGA POLICE DEPARTMENT
#TN0330100

#department dataframe
tn_crime = crime_all_df.loc[crime_all_df["ORI"]=="TN0330100"]

#changing index to offense and removing title
tn_crime = tn_crime.set_index("Offense")
tn_crime.index.name = None

#dropping human_trafficking & rape categoties
tn_crime = tn_crime.drop("human-trafficing")
tn_crime = tn_crime.drop("rape-legacy")

#get total number of crimes for the department
total_tn = tn_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
tn_crime = pd.DataFrame(tn_crime).T

#removing rows as it is duplicating data
tn_crime = tn_crime.drop("State")
tn_crime = tn_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_tn = (tn_crime["aggravated-assault"]["Crimes Committed"] + tn_crime["homicide"]["Crimes Committed"] 
            + tn_crime["rape"]["Crimes Committed"] + tn_crime["violent-crime"]["Crimes Committed"])
property_tn = (tn_crime["arson"]["Crimes Committed"] + tn_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + tn_crime["burglary"]["Crimes Committed"] + tn_crime["larceny"]["Crimes Committed"] 
            + tn_crime["property-crime"]["Crimes Committed"] + tn_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
tn_crime["ORI"] = "TN0330100"
tn_crime["State"] = "TN"
tn_crime["PD Name"] = "CHATTANOOGA POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
tn_crime = tn_crime.set_index("PD Name")
tn_crime.index.name = None

#sorting columns
tn_crime = tn_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
tn_crime = tn_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
tn_crime["Total Crime"] = total_tn
#sum of all person crimes
tn_crime["Total Person Crime"] = person_tn
#sum of all property crimes
tn_crime["Total Property Crime"] = property_tn

tn_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
CHATTANOOGA POLICE DEPARTMENT,TN,TN0330100,1556,25,1314,1098,33,7694,10106,196,161,2080,24263,3830,20433


In [18]:
#making second data frame transposed 
tn_crime2 = pd.DataFrame(tn_crime).T

#sending both to csv files so both options are available
tn_crime.to_csv("CSV_Files/PD_Crime_Data/TN0330100.csv", encoding="utf-8")
tn_crime2.to_csv("CSV_Files/PD_Crime_Data/TN0330100-2.csv", encoding="utf-8")

In [19]:
#station 4 of 12
#CHICAGO POLICE DEPT 
#ILCPD0000

#department dataframe
il_crime = crime_all_df.loc[crime_all_df["ORI"]=="ILCPD0000"]

#changing index to offense and removing title
il_crime = il_crime.set_index("Offense")
il_crime.index.name = None

#dropping human_trafficking & rape categoties
il_crime = il_crime.drop("human-trafficing")
il_crime = il_crime.drop("rape-legacy")

#get total number of crimes for the department
total_il = il_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
il_crime = pd.DataFrame(il_crime).T

#removing rows as it is duplicating data
il_crime = il_crime.drop("State")
il_crime = il_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_il = (il_crime["aggravated-assault"]["Crimes Committed"] + il_crime["homicide"]["Crimes Committed"] 
            + il_crime["rape"]["Crimes Committed"] + il_crime["violent-crime"]["Crimes Committed"])
property_il = (il_crime["arson"]["Crimes Committed"] + il_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + il_crime["burglary"]["Crimes Committed"] + il_crime["larceny"]["Crimes Committed"] 
            + il_crime["property-crime"]["Crimes Committed"] + il_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
il_crime["ORI"] = "ILCPD0000"
il_crime["State"] = "IL"
il_crime["PD Name"] = "CHICAGO POLICE DEPT" 

#make the PD name the index and dropping index name
il_crime = il_crime.set_index("PD Name")
il_crime.index.name = None

#sorting columns
il_crime = il_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
il_crime = il_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
il_crime["Total Crime"] = total_il
#sum of all person crimes
il_crime["Total Person Crime"] = person_il
#sum of all property crimes
il_crime["Total Property Crime"] = property_il

il_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
CHICAGO POLICE DEPT,IL,ILCPD0000,15296,0,9081,9578,492,62083,80742,7983,1761,25532,212548,43081,169467


In [20]:
#making second data frame transposed 
il_crime2 = pd.DataFrame(il_crime).T

#sending both to csv files so both options are available
il_crime.to_csv("CSV_Files/PD_Crime_Data/ILCPD0000.csv", encoding="utf-8")
il_crime2.to_csv("CSV_Files/PD_Crime_Data/ILCPD0000-2.csv", encoding="utf-8")

In [21]:
#station 5 of 12
#DALLAS POLICE DEPARTMENT 
#TXDPD0000

#department dataframe
tx_crime = crime_all_df.loc[crime_all_df["ORI"]=="TXDPD0000"]

#changing index to offense and removing title
tx_crime = tx_crime.set_index("Offense")
tx_crime.index.name = None

#dropping human_trafficking & rape categoties
tx_crime = tx_crime.drop("human-trafficing")
tx_crime = tx_crime.drop("rape-legacy")

#get total number of crimes for the department
total_tx = tx_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
tx_crime = pd.DataFrame(tx_crime).T

#removing rows as it is duplicating data
tx_crime = tx_crime.drop("State")
tx_crime = tx_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_tx = (tx_crime["aggravated-assault"]["Crimes Committed"] + tx_crime["homicide"]["Crimes Committed"] 
            + tx_crime["rape"]["Crimes Committed"] + tx_crime["violent-crime"]["Crimes Committed"])
property_tx = (tx_crime["arson"]["Crimes Committed"] + tx_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + tx_crime["burglary"]["Crimes Committed"] + tx_crime["larceny"]["Crimes Committed"] 
            + tx_crime["property-crime"]["Crimes Committed"] + tx_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
tx_crime["ORI"] = "TXDPD0000"
tx_crime["State"] = "TX"
tx_crime["PD Name"] = "DALLAS POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
tx_crime = tx_crime.set_index("PD Name")
tx_crime.index.name = None

#sorting columns
tx_crime = tx_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
tx_crime = tx_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
tx_crime["Total Crime"] = total_tx
#sum of all person crimes
tx_crime["Total Person Crime"] = person_tx
#sum of all property crimes
tx_crime["Total Property Crime"] = property_tx

tx_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
DALLAS POLICE DEPARTMENT,TX,TXDPD0000,6369,144,10257,9210,198,25812,45279,4400,797,12443,114909,19807,95102


In [22]:
#making second data frame transposed 
tx_crime2 = pd.DataFrame(tx_crime).T

#sending both to csv files so both options are available
tx_crime.to_csv("CSV_Files/PD_Crime_Data/TXDPD0000.csv", encoding="utf-8")
tx_crime2.to_csv("CSV_Files/PD_Crime_Data/TXDPD0000-2.csv", encoding="utf-8")

In [23]:
#station 6 of 12
#DENVER POLICE DEPARTMENT
#CODPD0000

#department dataframe
co_crime = crime_all_df.loc[crime_all_df["ORI"]=="CODPD0000"]

#changing index to offense and removing title
co_crime = co_crime.set_index("Offense")
co_crime.index.name = None

#dropping human_trafficking & rape categoties
co_crime = co_crime.drop("human-trafficing")
co_crime = co_crime.drop("rape-legacy")

#get total number of crimes for the department
total_co = co_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
co_crime = pd.DataFrame(co_crime).T

#removing rows as it is duplicating data
co_crime = co_crime.drop("State")
co_crime = co_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_co = (co_crime["aggravated-assault"]["Crimes Committed"] + co_crime["homicide"]["Crimes Committed"] 
            + co_crime["rape"]["Crimes Committed"] + co_crime["violent-crime"]["Crimes Committed"])
property_co = (co_crime["arson"]["Crimes Committed"] + co_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + co_crime["burglary"]["Crimes Committed"] + co_crime["larceny"]["Crimes Committed"] 
            + co_crime["property-crime"]["Crimes Committed"] + co_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
co_crime["ORI"] = "CODPD0000"
co_crime["State"] = "CO"
co_crime["PD Name"] = "DENVER POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
co_crime = co_crime.set_index("PD Name")
co_crime.index.name = None

#sorting columns
co_crime = co_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
co_crime = co_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
co_crime["Total Crime"] = total_co
#sum of all person crimes
co_crime["Total Person Crime"] = person_co
#sum of all property crimes
co_crime["Total Property Crime"] = property_co

co_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
DENVER POLICE DEPARTMENT,CO,CODPD0000,3474,107,5294,3967,67,18027,27288,1205,713,5932,66074,10186,55888


In [24]:
#making second data frame transposed 
co_crime2 = pd.DataFrame(co_crime).T

#sending both to csv files so both options are available
co_crime.to_csv("CSV_Files/PD_Crime_Data/CODPD0000.csv", encoding="utf-8")
co_crime2.to_csv("CSV_Files/PD_Crime_Data/CODPD0000-2.csv", encoding="utf-8")

In [25]:
#station 7 of 12
#KANSAS CITY POLICE DEPARTMENT 
#MOKPD0000

#department dataframe
mo_crime = crime_all_df.loc[crime_all_df["ORI"]=="MOKPD0000"]

#changing index to offense and removing title
mo_crime = mo_crime.set_index("Offense")
mo_crime.index.name = None

#dropping human_trafficking & rape categoties
mo_crime = mo_crime.drop("human-trafficing")
mo_crime = mo_crime.drop("rape-legacy")

#get total number of crimes for the department
total_mo = mo_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
mo_crime = pd.DataFrame(mo_crime).T

#removing rows as it is duplicating data
mo_crime = mo_crime.drop("State")
mo_crime = mo_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_mo = (mo_crime["aggravated-assault"]["Crimes Committed"] + mo_crime["homicide"]["Crimes Committed"] 
            + mo_crime["rape"]["Crimes Committed"] + mo_crime["violent-crime"]["Crimes Committed"])
property_mo = (mo_crime["arson"]["Crimes Committed"] + mo_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + mo_crime["burglary"]["Crimes Committed"] + mo_crime["larceny"]["Crimes Committed"] 
            + mo_crime["property-crime"]["Crimes Committed"] + mo_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
mo_crime["ORI"] = "MOKPD0000"
mo_crime["State"] = "MO"
mo_crime["PD Name"] = "KANSAS CITY POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
mo_crime = mo_crime.set_index("PD Name")
mo_crime.index.name = None

#sorting columns
mo_crime = mo_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
mo_crime = mo_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
mo_crime["Total Crime"] = total_mo
#sum of all person crimes
mo_crime["Total Person Crime"] = person_mo
#sum of all property crimes
mo_crime["Total Property Crime"] = property_mo

mo_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
KANSAS CITY POLICE DEPARTMENT,MO,MOKPD0000,5159,160,3801,3070,150,12253,19124,1443,347,7308,52815,12964,39851


In [26]:
#making second data frame transposed 
mo_crime2 = pd.DataFrame(mo_crime).T

#sending both to csv files so both options are available
mo_crime.to_csv("CSV_Files/PD_Crime_Data/MOKPD0000.csv", encoding="utf-8")
mo_crime2.to_csv("CSV_Files/PD_Crime_Data/MOKPD0000-2.csv", encoding="utf-8")

In [27]:
#station 8 of 12
#LAS VEGAS METRO POLICE DEPARTMENT
#NV0020100

#department dataframe
nv_crime = crime_all_df.loc[crime_all_df["ORI"]=="NV0020100"]

#changing index to offense and removing title
nv_crime = nv_crime.set_index("Offense")
nv_crime.index.name = None

#dropping human_trafficking & rape categoties
nv_crime = nv_crime.drop("human-trafficing")
nv_crime = nv_crime.drop("rape-legacy")

#get total number of crimes for the department
total_nv = nv_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
nv_crime = pd.DataFrame(nv_crime).T

#removing rows as it is duplicating data
nv_crime = nv_crime.drop("State")
nv_crime = nv_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_nv = (nv_crime["aggravated-assault"]["Crimes Committed"] + nv_crime["homicide"]["Crimes Committed"] 
            + nv_crime["rape"]["Crimes Committed"] + nv_crime["violent-crime"]["Crimes Committed"])
property_nv = (nv_crime["arson"]["Crimes Committed"] + nv_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + nv_crime["burglary"]["Crimes Committed"] + nv_crime["larceny"]["Crimes Committed"] 
            + nv_crime["property-crime"]["Crimes Committed"] + nv_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
nv_crime["ORI"] = "NV0020100"
nv_crime["State"] = "NV"
nv_crime["PD Name"] = "LAS VEGAS METRO POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
nv_crime = nv_crime.set_index("PD Name")
nv_crime.index.name = None

#sorting columns
nv_crime = nv_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
nv_crime = nv_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson", 
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
nv_crime["Total Crime"] = total_nv
#sum of all person crimes
nv_crime["Total Person Crime"] = person_nv
#sum of all property crimes
nv_crime["Total Property Crime"] = property_nv

nv_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
LAS VEGAS METRO POLICE DEPARTMENT,NV,NV0020100,5213,98,7311,10646,84,28240,46197,2118,1439,8854,110200,15590,94610


In [28]:
#making second data frame transposed 
nv_crime2 = pd.DataFrame(nv_crime).T

#sending both to csv files so both options are available
nv_crime.to_csv("CSV_Files/PD_Crime_Data/NV0020100.csv", encoding="utf-8")
nv_crime2.to_csv("CSV_Files/PD_Crime_Data/NV0020100-2.csv", encoding="utf-8")

In [29]:
#station 9 of 12
#NEW ORLEANS POLICE DEPARTMENT 
#LANPD0000

#department dataframe
la_crime = crime_all_df.loc[crime_all_df["ORI"]=="LANPD0000"]

#changing index to offense and removing title
la_crime = la_crime.set_index("Offense")
la_crime.index.name = None

#dropping human_trafficking & rape categoties
la_crime = la_crime.drop("human-trafficing")
la_crime = la_crime.drop("rape-legacy")

#get total number of crimes for the department
total_la = la_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
la_crime = pd.DataFrame(la_crime).T

#removing rows as it is duplicating data
la_crime = la_crime.drop("State")
la_crime = la_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_la = (la_crime["aggravated-assault"]["Crimes Committed"] + la_crime["homicide"]["Crimes Committed"] 
            + la_crime["rape"]["Crimes Committed"] + la_crime["violent-crime"]["Crimes Committed"])
property_la = (la_crime["arson"]["Crimes Committed"] + la_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + la_crime["burglary"]["Crimes Committed"] + la_crime["larceny"]["Crimes Committed"] 
            + la_crime["property-crime"]["Crimes Committed"] + la_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
la_crime["ORI"] = "LANPD0000"
la_crime["State"] = "LA"
la_crime["PD Name"] = "NEW ORLEANS POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
la_crime = la_crime.set_index("PD Name")
la_crime.index.name = None

#sorting columns
la_crime = la_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
la_crime = la_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
la_crime["Total Crime"] = total_la
#sum of all person crimes
la_crime["Total Person Crime"] = person_la
#sum of all property crimes
la_crime["Total Property Crime"] = property_la

la_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
NEW ORLEANS POLICE DEPARTMENT,LA,LANPD0000,2608,0,2951,2143,121,15785,20879,1013,774,4516,50790,8019,42771


In [30]:
#making second data frame transposed 
la_crime2 = pd.DataFrame(la_crime).T

#sending both to csv files so both options are available
la_crime.to_csv("CSV_Files/PD_Crime_Data/LANPD0000.csv", encoding="utf-8")
la_crime2.to_csv("CSV_Files/PD_Crime_Data/LANPD0000-2.csv", encoding="utf-8")

In [31]:
#station 10 of 12
#PORTLAND POLICE BUREAU 
#OR0260200

#department dataframe
or_crime = crime_all_df.loc[crime_all_df["ORI"]=="OR0260200"]

#changing index to offense and removing title
or_crime = or_crime.set_index("Offense")
or_crime.index.name = None

#dropping human_trafficking & rape categoties
or_crime = or_crime.drop("human-trafficing")
or_crime = or_crime.drop("rape-legacy")

#get total number of crimes for the department
total_or = or_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
or_crime = pd.DataFrame(or_crime).T

#removing rows as it is duplicating data
or_crime = or_crime.drop("State")
or_crime = or_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_or = (or_crime["aggravated-assault"]["Crimes Committed"] + or_crime["homicide"]["Crimes Committed"]
            + or_crime["rape"]["Crimes Committed"] + or_crime["violent-crime"]["Crimes Committed"])
property_or = (or_crime["arson"]["Crimes Committed"] + or_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + or_crime["burglary"]["Crimes Committed"] + or_crime["larceny"]["Crimes Committed"] 
            + or_crime["property-crime"]["Crimes Committed"] + or_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
or_crime["ORI"] = "OR0260200"
or_crime["State"] = "OR"
or_crime["PD Name"] = "PORTLAND POLICE BUREAU" 

#make the PD name the index and dropping index name
or_crime = or_crime.set_index("PD Name")
or_crime.index.name = None

#sorting columns
or_crime = or_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
or_crime = or_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
or_crime["Total Crime"] = total_or
#sum of all person crimes
or_crime["Total Person Crime"] = person_or
#sum of all property crimes
or_crime["Total Property Crime"] = property_or

or_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
PORTLAND POLICE BUREAU,OR,OR0260200,2230,252,6432,4200,29,23820,34452,979,368,3879,76641,6506,70135


In [32]:
#making second data frame transposed 
or_crime2 = pd.DataFrame(or_crime).T

#sending both to csv files so both options are available
or_crime.to_csv("CSV_Files/PD_Crime_Data/OR0260200.csv", encoding="utf-8")
or_crime2.to_csv("CSV_Files/PD_Crime_Data/OR0260200-2.csv", encoding="utf-8")

In [33]:
#station 11 of 12
#SAN FRANCISCO POLICE DEPARTMENT
#CA0380100

#department dataframe
ca_crime = crime_all_df.loc[crime_all_df["ORI"]=="CA0380100"]

#changing index to offense and removing title
ca_crime = ca_crime.set_index("Offense")
ca_crime.index.name = None

#dropping human_trafficking & rape categoties
ca_crime = ca_crime.drop("human-trafficing")
ca_crime = ca_crime.drop("rape-legacy")

#get total number of crimes for the department
total_ca = ca_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
ca_crime = pd.DataFrame(ca_crime).T

#removing rows as it is duplicating data
ca_crime = ca_crime.drop("State")
ca_crime = ca_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_ca = (ca_crime["aggravated-assault"]["Crimes Committed"] + ca_crime["homicide"]["Crimes Committed"]
            + ca_crime["rape"]["Crimes Committed"] + ca_crime["violent-crime"]["Crimes Committed"])
property_ca = (ca_crime["arson"]["Crimes Committed"] + ca_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + ca_crime["burglary"]["Crimes Committed"] + ca_crime["larceny"]["Crimes Committed"] 
            + ca_crime["property-crime"]["Crimes Committed"] + ca_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
ca_crime["ORI"] = "CA0380100"
ca_crime["State"] = "CA"
ca_crime["PD Name"] = "SAN FRANCISCO POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
ca_crime = ca_crime.set_index("PD Name")
ca_crime.index.name = None

#sorting columns
ca_crime = ca_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
ca_crime = ca_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
ca_crime["Total Crime"] = total_ca
#sum of all person crimes
ca_crime["Total Person Crime"] = person_ca
#sum of all property crimes
ca_crime["Total Property Crime"] = property_ca

ca_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
SAN FRANCISCO POLICE DEPARTMENT,CA,CA0380100,2514,56,4249,4644,40,39887,48780,3055,324,5933,109482,8811,100671


In [34]:
#making second data frame transposed 
ca_crime2 = pd.DataFrame(ca_crime).T

#sending both to csv files so both options are available
ca_crime.to_csv("CSV_Files/PD_Crime_Data/CA0380100.csv", encoding="utf-8")
ca_crime2.to_csv("CSV_Files/PD_Crime_Data/CA0380100-2.csv", encoding="utf-8")

In [35]:
#station 12 of 12
#SIOUX FALLS POLICE DEPARTMENT
#SD0490200

#department dataframe
sd_crime = crime_all_df.loc[crime_all_df["ORI"]=="SD0490200"]

#changing index to offense and removing title
sd_crime = sd_crime.set_index("Offense")
sd_crime.index.name = None

#dropping human_trafficking & rape categoties
sd_crime = sd_crime.drop("human-trafficing")
sd_crime = sd_crime.drop("rape-legacy")

#get total number of crimes for the department
total_sd = sd_crime["Crimes Committed"].sum()

#transposing to make the offenses the columns
sd_crime = pd.DataFrame(sd_crime).T

#removing rows as it is duplicating data
sd_crime = sd_crime.drop("State")
sd_crime = sd_crime.drop("ORI")

#get two different sums, one for person crimes, and one for property crimes
person_sd = (sd_crime["aggravated-assault"]["Crimes Committed"] + sd_crime["homicide"]["Crimes Committed"]
            + sd_crime["rape"]["Crimes Committed"] + sd_crime["violent-crime"]["Crimes Committed"])
property_sd = (sd_crime["arson"]["Crimes Committed"] + sd_crime["motor-vehicle-theft"]["Crimes Committed"] 
            + sd_crime["burglary"]["Crimes Committed"] + sd_crime["larceny"]["Crimes Committed"] 
            + sd_crime["property-crime"]["Crimes Committed"] + sd_crime["robbery"]["Crimes Committed"])


#adding department name, ORI & State as columns
sd_crime["ORI"] = "SD0490200"
sd_crime["State"] = "SD"
sd_crime["PD Name"] = "SIOUX FALLS POLICE DEPARTMENT" 

#make the PD name the index and dropping index name
sd_crime = sd_crime.set_index("PD Name")
sd_crime.index.name = None

#sorting columns
sd_crime = sd_crime[["State","ORI","aggravated-assault", "arson","motor-vehicle-theft", "burglary", 
                     "homicide","larceny","property-crime","robbery","rape","violent-crime"]]
#renaming columns - note that rape was changed to sex crimes, and motor-vehicle theft was changed to auto theft
sd_crime = sd_crime.rename(columns = {"State":"State","ORI":"ORI", "aggravated-assault":"Agg Assault", 
                                      "motor-vehicle-theft":"Auto Theft", "arson":"Arson",
                                      "burglary":"Burglary", "homicide":"Homicide","larceny":"Larceny",
                                      "property-crime":"Property Crime","rape":"Sex Crimes",
                                      "robbery":"Robbery","violent-crime":"Violent Crime"})


        
#adding columns for totals to data frame
#sums for all crimes
sd_crime["Total Crime"] = total_sd
#sum of all person crimes
sd_crime["Total Person Crime"] = person_sd
#sum of all property crimes
sd_crime["Total Property Crime"] = property_sd

sd_crime

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
SIOUX FALLS POLICE DEPARTMENT,SD,SD0490200,709,29,658,695,4,4300,5653,68,116,998,13230,1827,11403


In [36]:
#making second data frame transposed 
sd_crime2 = pd.DataFrame(sd_crime).T

#sending both to csv files so both options are available
sd_crime.to_csv("CSV_Files/PD_Crime_Data/SD0490200.csv", encoding="utf-8")
sd_crime2.to_csv("CSV_Files/PD_Crime_Data/SD0490200-2.csv", encoding="utf-8")

In [37]:
#APPEND ALL DEPARTMETNS TOGETHER

total_crime_all = ma_crime.append(va_crime)
total_crime_all = total_crime_all.append(tn_crime)
total_crime_all = total_crime_all.append(il_crime)
total_crime_all = total_crime_all.append(tx_crime)
total_crime_all = total_crime_all.append(co_crime)
total_crime_all = total_crime_all.append(mo_crime)
total_crime_all = total_crime_all.append(nv_crime)
total_crime_all = total_crime_all.append(la_crime)
total_crime_all = total_crime_all.append(or_crime)
total_crime_all = total_crime_all.append(ca_crime)
total_crime_all = total_crime_all.append(sd_crime)
total_crime_all

,State,ORI,Agg Assault,Arson,Auto Theft,Burglary,Homicide,Larceny,Property Crime,Robbery,Sex Crimes,Violent Crime,Total Crime,Total Person Crime,Total Property Crime
BOSTON POLICE DEPARTMENT,MA,MA0130100,2932,4,920,1703,42,10590,13213,1039,231,4284,34958,7489,27469
CHARLOTTESVILLE POLICE DEPT,VA,VA1020000,103,5,78,122,0,924,1124,22,32,181,2591,316,2275
CHATTANOOGA POLICE DEPARTMENT,TN,TN0330100,1556,25,1314,1098,33,7694,10106,196,161,2080,24263,3830,20433
CHICAGO POLICE DEPT,IL,ILCPD0000,15296,0,9081,9578,492,62083,80742,7983,1761,25532,212548,43081,169467
DALLAS POLICE DEPARTMENT,TX,TXDPD0000,6369,144,10257,9210,198,25812,45279,4400,797,12443,114909,19807,95102
DENVER POLICE DEPARTMENT,CO,CODPD0000,3474,107,5294,3967,67,18027,27288,1205,713,5932,66074,10186,55888
KANSAS CITY POLICE DEPARTMENT,MO,MOKPD0000,5159,160,3801,3070,150,12253,19124,1443,347,7308,52815,12964,39851
LAS VEGAS METRO POLICE DEPARTMENT,NV,NV0020100,5213,98,7311,10646,84,28240,46197,2118,1439,8854,110200,15590,94610
NEW ORLEANS POLICE DEPARTMENT,LA,LANPD0000,2608,0,2951,2143,121,15785,20879,1013,774,4516,50790,8019,42771
PORTLAND POLICE BUREAU,OR,OR0260200,2230,252,6432,4200,29,23820,34452,979,368,3879,76641,6506,70135


In [38]:
#making second data frame transposed 
total_crime_all2 = pd.DataFrame(total_crime_all).T

#sending both to csv files so both options are available
total_crime_all.to_csv("CSV_Files/PD_Crime_Data/ALL_CRIME.csv", encoding="utf-8")
total_crime_all2.to_csv("CSV_Files/PD_Crime_Data/ALL_CRIME2.csv", encoding="utf-8")